In [1]:
import pandas as pd
import numpy as np

In [2]:
# df  = pd.read_csv('../data/hawkes_analysis_data.csv')
df  = pd.read_csv(r'I:/Hawkes_Model_MM/hawkes_analysis_data.csv')

In [3]:
df.head()

,trade_number,trade_price,volume,buy_order_number,buy_algo,buy_client,sell_order_number,sell_algo,sell_client,timestamp,buy_entry_ts,sell_entry_ts,aggressor_side,date
0,2019081325008066,785.0,1,1100000000068964,1,2,1100000000062580,1,3,2019-08-13 09:15:00.006118774,2019-08-13 09:15:00.005767822,NaN,-1,2019-08-13
1,2019081325008128,785.0,2,1100000000069441,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024169922,2019-08-13 09:15:00.024139404,NaN,-1,2019-08-13
2,2019081325008129,785.0,47,1100000000069442,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024230957,2019-08-13 09:15:00.024215698,NaN,-1,2019-08-13
3,2019081325008130,785.0,53,1100000000069442,1,3,1100000000068983,0,3,2019-08-13 09:15:00.024246216,2019-08-13 09:15:00.024215698,2019-08-13 09:15:00.005935669,1,2019-08-13
4,2019081325008466,785.0,10,1100000000055032,1,3,1100000000070554,0,1,2019-08-13 09:15:00.118347168,NaN,2019-08-13 09:15:00.118331909,1,2019-08-13


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
dates = df['date'].unique()

In [6]:
target_date = dates[0]

In [7]:
# df_day = df[df['date'] == target_date].sort_values('timestamp')
df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])

In [8]:
df_day['timestamp'].dtype

dtype('<M8[ns]')

In [9]:
market_open = pd.to_datetime((target_date) + ' 09:15:00')
df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()

In [10]:
buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()

events = [buy_times, sell_times]

In [11]:
decay = 1/60 

In [12]:
from BivariateHawkes_optimized import BivariateHawkesOptimized

Numba not available. Install with: pip install numba


In [13]:
model = BivariateHawkesOptimized()

In [14]:
model.fit(buy_times, sell_times, optimize_beta=True)

In [15]:
results = model.get_results()

In [16]:
results

{'mu_buy': np.float64(1e-10),
 'mu_sell': np.float64(1.5922560349197657e-05),
 'alpha_bb': np.float64(11110.233936482926),
 'alpha_bs': np.float64(4328.096228617026),
 'alpha_sb': np.float64(3885.0443986418936),
 'alpha_ss': np.float64(9725.6813823859),
 'beta': np.float64(24563.963992568843),
 'br_bb': np.float64(0.4522980875498769),
 'br_bs': np.float64(0.17619697822087566),
 'br_sb': np.float64(0.15816031971945602),
 'br_ss': np.float64(0.3959328952496486),
 'spectral_radius': np.float64(0.5934129489553213),
 'total_branching': np.float64(1.1825882807398571),
 'log_likelihood': np.float64(1470278.0284006584),
 'is_stationary': np.True_}

In [20]:
pd.DataFrame({target_date: results}).T

,mu_buy,mu_sell,alpha_bb,alpha_bs,alpha_sb,alpha_ss,beta,br_bb,br_bs,br_sb,br_ss,spectral_radius,total_branching,log_likelihood,is_stationary
2019-08-13,0.0,0.000016,11110.233936,4328.096229,3885.044399,9725.681382,24563.963993,0.452298,0.176197,0.15816,0.395933,0.593413,1.182588,1470278.028401,True


In [18]:
buy_times

array([2.42462160e-02, 1.18347168e-01, 3.17398071e-01, ...,
       2.42857539e+04, 2.42857539e+04, 2.42857539e+04])

In [21]:
all_day_results = {target_date: results}

In [22]:
all_day_results

{'2019-08-13': {'mu_buy': np.float64(1e-10),
  'mu_sell': np.float64(1.5922560349197657e-05),
  'alpha_bb': np.float64(11110.233936482926),
  'alpha_bs': np.float64(4328.096228617026),
  'alpha_sb': np.float64(3885.0443986418936),
  'alpha_ss': np.float64(9725.6813823859),
  'beta': np.float64(24563.963992568843),
  'br_bb': np.float64(0.4522980875498769),
  'br_bs': np.float64(0.17619697822087566),
  'br_sb': np.float64(0.15816031971945602),
  'br_ss': np.float64(0.3959328952496486),
  'spectral_radius': np.float64(0.5934129489553213),
  'total_branching': np.float64(1.1825882807398571),
  'log_likelihood': np.float64(1470278.0284006584),
  'is_stationary': np.True_}}

In [23]:
for target_date in dates[1:]:
    df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])
    market_open = pd.to_datetime((target_date) + ' 09:15:00')
    df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()
    buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
    sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()
    events = [buy_times, sell_times]
    model = BivariateHawkesOptimized()
    model.fit(buy_times, sell_times, optimize_beta=True)
    r = model.get_results()
    all_day_results[target_date] = r
pd.DataFrame(all_day_results).T.to_csv('../data/all_day_results.csv')

OSError: Cannot save file into a non-existent directory: '..\data'

In [26]:
pd.DataFrame(all_day_results).T.to_csv('../data/all_day_results.csv')

In [27]:
pd.DataFrame(all_day_results).T.to_csv('all_day_results_{}.csv'.format(target_date))

In [29]:
from MarkedBivariateHawkes_optimized import MarkedBivariateHawkesOptimized

In [31]:
# Cell: Process all dates with marked Hawkes (similar to all_day_results)
all_day_marked_results = {}

for i, target_date in enumerate(dates):
    print(f"Processing {i+1}/{len(dates)}: {target_date}...")
    
    # Prepare data for the day
    df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])
    market_open = pd.to_datetime((target_date) + ' 09:15:00')
    df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()
    
    # Extract buy and sell events with volumes
    buy_mask = df_day['aggressor_side'] == +1
    sell_mask = df_day['aggressor_side'] == -1
    
    buy_times = df_day[buy_mask]['t'].to_numpy()
    sell_times = df_day[sell_mask]['t'].to_numpy()
    buy_volumes = df_day[buy_mask]['volume'].to_numpy()
    sell_volumes = df_day[sell_mask]['volume'].to_numpy()
    
    # Skip if insufficient data
    if len(buy_times) < 10 or len(sell_times) < 10:
        print(f"  Skipping: insufficient events")
        continue
    
    try:
        # Fit marked Hawkes model
        model = MarkedBivariateHawkesOptimized()
        model.fit(buy_times, sell_times, buy_volumes, sell_volumes, 
                  optimize_beta=True, verbose=False)
        
        # Store results
        all_day_marked_results[target_date] = model.get_results()
        pd.DataFrame(all_day_marked_results).T.to_csv('all_day_marked_results_{}.csv'.format(target_date))
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

print(f"\nCompleted: {len(all_day_marked_results)}/{len(dates)} dates")

# Cell: Save results as DataFrame
df_marked_results = pd.DataFrame(all_day_marked_results).T
df_marked_results.to_csv('../data/all_day_marked_results.csv')
print(f"Saved to: ../data/all_day_marked_results.c
sv")
print(df_marked_results.head())

Processing 1/10: 2019-08-13...
Processing 2/10: 2019-08-14...
Processing 3/10: 2019-08-16...
Processing 4/10: 2019-08-19...
Processing 5/10: 2019-08-20...
Processing 6/10: 2019-08-21...
Processing 7/10: 2019-08-22...
Processing 8/10: 2019-08-23...
Processing 9/10: 2019-08-26...
Processing 10/10: 2019-08-27...

Completed: 10/10 dates
Saved to: ../data/all_day_marked_results.csv
              mu_buy   mu_sell      alpha_bb      alpha_bs     alpha_sb  \
2019-08-13       0.0   0.00005   1413.224351   1351.644465   484.358717   
2019-08-14  0.000058       0.0   2972.043103    603.450777   647.627549   
2019-08-16  0.001694       0.0   1399.760427    289.769062   643.928139   
2019-08-19       0.0  0.001814   2503.635706    751.308251  1012.084692   
2019-08-20  0.030899       0.0  25357.324177  10285.932356  4820.213991   

                alpha_ss          beta  avg_mark     br_bb     br_bs  \
2019-08-13   1569.618928  10376.059644  2.716042  0.369926  0.353807   
2019-08-14   3214.557979 